## Install Libraries

In [ ]:
!pip install gradio
!pip install librosa
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install pillow
!pip install tensorflow tensorflow_hub

## Import Libraries

In [ ]:
import gradio as gr
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import random
import tensorflow as tf
import tensorflow_hub as hub

## Audio Processing and Neural Style Transfer

In [ ]:
# Style images and their paths (please change to correct paths)
style_images = {
    "Starry Night": "Desktop/Echoes_of_Art/Paintings/StarryNight.jpg",
    "The Scream": "Desktop/Echoes_of_Art/Paintings/Scream.jpg",
    "The Great Wave": "Desktop/Echoes_of_Art/Paintings/Wave.jpg",
    "Impression, Sunrise": "Desktop/Echoes_of_Art/Paintings/Sunrise.jpg",
    "Persistence of Memory": "Desktop/Echoes_of_Art/Paintings/PersistenceofMemory.jpg",
}

# Load style transfer model
hub_model = hub.load("https://www.kaggle.com/models/google/arbitrary-image-stylization-v1/TensorFlow1/256/2")

#Load and process image function
def load_image(image_path, image_size=(256, 256)):
    try:
        img = Image.open(image_path).convert("RGB")
        img = img.resize(image_size)
        img = np.array(img) / 255.0

        # Add noise (value between 0.05 and 0.08)
        noise_level = np.random.uniform(0.05, 0.08)
        noise = np.random.normal(0, noise_level, img.shape)
        img = np.clip(img + noise, 0, 1)

        # Apply glitch effect randomly (shift between -20 to 20 pixels, random axis chosen)
        shift = random.randint(-20, 20)
        axis = random.choice([0, 1])
        img = np.roll(img, shift, axis=axis)

        # Apply ghosting effect (shift between value 2 and 10, transparency level between 0.3 and 0.6, random direction (horizonal, vertical))
        ghost_shift = random.randint(2, 6)
        ghost_alpha = random.uniform(0.3, 0.6)
        ghost_axis = random.choice([0, 1])
        ghost_img = np.roll(img, ghost_shift, axis=ghost_axis)
        img = np.clip(img * (1 - ghost_alpha) + ghost_img * ghost_alpha, 0, 1)

        return np.expand_dims(img, axis=0)

    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return np.zeros((1, *image_size, 3))

# convert audio to spectrogram function
def audio_to_spectrogram(audio_input):
    try:
        # audio handling
        if isinstance(audio_input, str):
            y, sr = librosa.load(audio_input, sr=None)
        elif hasattr(audio_input, 'name'):
            y, sr = librosa.load(audio_input.name, sr=None)
        else:
            sr, y = audio_input
            y = y.astype(np.float32) / np.iinfo(y.dtype).max

        #to spectrogram using Short-Time Fourier Transform
        D = librosa.stft(y)
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

        fig, ax = plt.subplots(figsize=(10, 5))
        ax.set_axis_off()
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        librosa.display.specshow(S_db, sr=sr, cmap='inferno')

        img_path = "spectrogram.png"
        plt.savefig(img_path, dpi=300, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        return img_path
    except Exception as e:
        print(f"Error generating spectrogram: {e}")
        return None

# apply NST style function
def apply_nst_style(content_path, style_name):
    try:
        # Load images, content image = spectrogram
        content_img = load_image(content_path)

        # load style image, style image = paintings
        style_img = load_image(style_images[style_name])

        # Apply style transfer
        stylized = hub_model(
            tf.constant(content_img, dtype=tf.float32),
            tf.constant(style_img, dtype=tf.float32)
        )[0]

        # Convert to PIL Image
        stylized = np.squeeze(stylized.numpy(), axis=0)
        stylized = np.clip(stylized * 255, 0, 255).astype(np.uint8)
        stylized_img = Image.fromarray(stylized)

        # Enlarge the output image
        output_size = (1024, 1024)
        stylized_img = stylized_img.resize(output_size, Image.Resampling.LANCZOS)

        # Save result
        output_path = "stylized_spectrogram.png"
        stylized_img.save(output_path)

        return output_path

    except Exception as e:
        print(f"Error applying NST: {e}")
        return content_path  # Return original if error occurs

#gradio interface
with gr.Blocks(theme=gr.themes.Default(font=[gr.themes.GoogleFont("Roboto"), "Arial", "sans-serif"])) as demo:
    # Title and description
    gr.Markdown("<h1 style='font-size: 36px;'>Echoes of Art</h1>")
    gr.Markdown("<p style='font-size: 18px; padding-bottom: 20px;'>Upload or record your own audio and generate a unique visual piece inspired by a famous painting style using Neural Style Transfer.</p>")

    # Audio section
    with gr.Row(equal_height=True):
        with gr.Column(min_width=400):
            with gr.Tab("Upload Audio"):
                audio_upload = gr.File(label="Upload Audio File", file_types=["audio"])
                upload_button = gr.Button("Generate from Upload", variant="primary")

            with gr.Tab("Record Audio"):
                mic_recording = gr.Audio(label="Record with Microphone", sources=["microphone"], type="numpy")
                record_button = gr.Button("Generate from Recording", variant="primary")

        #Spectrogram image box
        with gr.Column():
            spectrogram_output = gr.Image(label="Original Spectrogram", height=300, interactive=False)

    # Generate spectrogram
    def process_audio(audio_input):
        spectrogram = audio_to_spectrogram(audio_input)
        return spectrogram if spectrogram else None

    upload_button.click(fn=process_audio, inputs=audio_upload, outputs=spectrogram_output)
    record_button.click(fn=process_audio, inputs=mic_recording, outputs=spectrogram_output)

    # line separator
    gr.HTML("<hr style='margin: 20px 0; border: 1px solid #ddd;'>")

    # Painting style selector
    gr.Markdown("## Select a Painting Style")
    with gr.Row():
        style_buttons = []
        for style_name, style_path in style_images.items():
            with gr.Column(min_width=200):
                btn = gr.Button(style_name, variant="primary")
                style_buttons.append(btn)
                thumbnail = Image.open(style_path).resize((500, 500))
                gr.Image(thumbnail, label=style_name, interactive=False, height=500, width=500)

    # line separator
    gr.HTML("<hr style='margin: 20px 0; border: 1px solid #ddd;'>")

    # Stylized result image box
    gr.Markdown("## Stylized Result")
    stylized_output = gr.Image(height=400)

    # Connect style buttons to output
    for btn, style_name in zip(style_buttons, style_images.keys()):
      btn.click(
        fn=apply_nst_style,
        inputs=[spectrogram_output, gr.Text(style_name, visible=False)],
        outputs=stylized_output
    )

#run interface
demo.launch()